In [ ]:
%reset -f
import numpy as np

#plotting
import matplotlib.pyplot as plt

In [ ]:
#import matplotlib.style as style
#style.use("grayscale")
%matplotlib inline

#units
from astropy import units as u

#POPPY
import poppy
import sys
from poppy import fresnel


from poppy.poppy_core import _PUPIL, _IMAGE, _INTERMED

#logging
import logging
_log = logging.getLogger('poppy')
_log.setLevel(logging.DEBUG)

#make logging work in IPYTHON:
try:
    from IPython.core.debugger import Tracer; stop = Tracer()
except:
    pass

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')



In [ ]:
class sinc_squared_occulter(poppy.AnalyticOpticalElement):

    def __init__(self, name='occulter', **kwargs):
        poppy.AnalyticOpticalElement.__init__(self, name=name, **kwargs)
        
    def getPhasor(self, wave):
        hwhm_sinc_squared = 1.7996
        hwhm = 2.5
        pupil_ratio = (1.0/wave.oversample)
        y, x = wave.coordinates()
        r = (np.sqrt(x**2 + y**2)/wave.pixelscale) * pupil_ratio * hwhm_sinc_squared/hwhm
        theta = np.arctan2(y,x)
        amp = 1 - (np.sinc(r/np.pi))**2 #divide by pi, b/c np.sinc=sin(x*pi)/x*pi
        return amp

In [ ]:
class phase_aberation_inducer(poppy.AnalyticOpticalElement):

    def __init__(self, name=None, radius=1.0, pad_factor=1.0, map_adjust = 1e5, **kwargs):
        try:
            self.radius = float(radius)
        except ValueError:
            raise TypeError("Argument 'radius' must be the radius of the pupil in meters")
            
        if name is None:
            name = "Circle, radius=%.2f m" % radius
        super(phase_aberation_inducer, self).__init__( name=name, planetype=_PUPIL, **kwargs)
        # for creating input wavefronts - let's pad a bit:
        self.pupil_diam = pad_factor * 2 * self.radius
        self.map_adjust = map_adjust

    def __load_aberation_map_interpolator__(self,radius):
        from scipy.interpolate import InterpolatedUnivariateSpline
        dataFile = 'new_circular_error_map.dat'
        fd = open(dataFile,'rb')
        fd.seek(0,0)
        nmap = np.fromfile(fd, dtype = np.long, count = 1)[0]
        fd.seek(4,0)
        rmap = np.fromfile(fd, dtype = np.float64, count = nmap) * radius
        zmap = np.fromfile(fd, dtype = np.float64, count = nmap)
        fd.close()
        f = InterpolatedUnivariateSpline(rmap, zmap, k=4)
        return f
    
    def getPhasor(self, wave):
        if not isinstance(wave, poppy.Wavefront):  # pragma: no cover
            raise ValueError("CircularAperture getPhasor must be called with a Wavefront "
                             "to define the spacing")
        assert (wave.planetype != _IMAGE)
        

        map_adjust = 1e5

        y, x = self.get_coordinates(wave)
        r = np.sqrt(x ** 2 + y ** 2)*wave.pixelscale
        del x
        del y
        interpolator = self.__load_aberation_map_interpolator__(self.radius)
        self.aberation_map = self.map_adjust*interpolator(r)
        w_outside = np.where(r > self.radius)
        del r
        self.transmission = np.ones(wave.shape) + np.exp(1.j * 2 * np.pi * self.aberation_map)
        self.transmission[w_outside] = 0
        return self.transmission

In [ ]:
def poppy_system(wavefront, use_occulter=True):
    
    diam = 0.025*u.m
    fl = 0.7*u.m
    
    pupil_ratio = (1.0/wavefront.oversample)
    
    wavefront *= poppy.CircularAperture(radius=diam.value/2.0)
    #wavefront *= phase_aberation_inducer(radius=diam.value/2.0)
    wavefront.apply_lens_power(fresnel.GaussianLens(fl, name='M1'))
    wavefront *= poppy.CircularAperture(radius=diam.value/2.0)
    

    wavefront.propagate_fresnel(fl,display_intermed=False)
    
    
    if (use_occulter): wavefront *= sinc_squared_occulter(name='Occulter')
#     wavefront *= poppy.CircularAperture(radius=250.0*pupil_ratio*wavefront.pixelscale)
    
    
    wavefront.propagate_fresnel(fl,display_intermed=False)
    
    
    wavefront.apply_lens_power(fresnel.GaussianLens(fl, name='M2'))
    wavefront *= poppy.CircularAperture(radius=1.6*diam.value/2)

    
    wavefront.propagate_fresnel(2*fl,display_intermed=False)

    
    wavefront *= poppy.CircularAperture(radius=0.53*diam.value/2, name='Lyot Stop')
    wavefront.apply_lens_power(fresnel.GaussianLens(fl, name='Lyot Stop'))
    
    
    wavefront.propagate_fresnel(fl,display_intermed=False)
    
    return wavefront

In [ ]:
#Using the same simple setup used in TDEM milestone 1 report, Appendix B: Verification of PROPER (p. 26)
#Ref Figure 9 for the setup, Ref p. 27 for dimensions

lam = 500e-9

osamp=8 #oversampling factor

n=256

beam_diameter = 0.025*u.m

#intialize wavefront
poppy_field = fresnel.FresnelWavefront(beam_radius=beam_diameter/2, wavelength=lam, npix=n, oversample=osamp)
poppy_psf = fresnel.FresnelWavefront(beam_radius=beam_diameter/2, wavelength=lam, npix=n, oversample=osamp)

poppy_field = poppy_system(poppy_field,use_occulter=True)
poppy_psf = poppy_system(poppy_psf,use_occulter=False)

poppy_field_profile = poppy_field.intensity[osamp*n/2:,osamp*n/2]
poppy_psf_profile = poppy_psf.intensity[osamp*n/2:,osamp*n/2]

poppy_contrast_profile = poppy_field_profile / poppy_psf_profile





plt.figure(figsize=(10,5))
plt.semilogy(poppy_field_profile)
plt.show()

plt.figure(figsize=(10,5))
plt.semilogy(poppy_psf_profile)
plt.show()

plt.figure(figsize=(10,5))
plt.semilogy(poppy_contrast_profile)
plt.show()






In [ ]:
plt.figure(figsize=(10,5))
plt.semilogy(poppy_field_profile)
plt.show()

plt.figure(figsize=(10,5))
plt.semilogy(poppy_psf_profile)
plt.show()

poppy_contrast_profile = poppy_field_profile / poppy_psf_profile

plt.figure(figsize=(10,5))
plt.semilogy(poppy_contrast_profile)
plt.show()